## <b><font color='darkblue'>Preface</font></b>
Here we are going to demo in using `langfun` to write unit test case.

### <b><font color='darkgreen'>Import packages</font></b>
Let's import necessary packages:

In [1]:
!pip freeze | grep -P "(langfun)"

langfun==0.1.1


In [2]:
import pathlib
import os
import langfun as lf
import pyglove as pg


Path = pathlib.Path
print("*" * len(os.environ.get('GOOGLE_API_KEY', '')))

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***************************************


In [3]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

## <b><font color='darkblue'>Natural Language -> Unit test case</font></b>
To let LLM to write unit test case for us, we need to initialize the LLM and provide the context.

### <b><font color='darkgreen'>Initialization</font></b>
Let's initialize our LLM and environment:

In [5]:
[model for model in dir(lf.llms) if 'Gemini' in model]

['GeminiPro',
 'GeminiPro1_5',
 'GeminiProVision',
 'VertexAIGeminiFlash1_5',
 'VertexAIGeminiFlash1_5_0514',
 'VertexAIGeminiPro1',
 'VertexAIGeminiPro1Vision',
 'VertexAIGeminiPro1_5',
 'VertexAIGeminiPro1_5_0409',
 'VertexAIGeminiPro1_5_0514']

In [4]:
model = lf.llms.GeminiPro1_5()

In [5]:
class TestModule(pg.Object):
    project_root_path: str
    module_package: str
    module_name: str
    module_content: str

class UnitTestCases(pg.Object):
    project_root_path: str
    module_package: str
    module_name: str
    test_case_content: str

    @property
    def test_module_name(self) -> str:
        return f'test_{self.module_name}.py'
    
    @property
    def unit_test_root_path(self) -> str:
        return str(
            Path(self.project_root_path) / Path('tests/unit'))

    @property
    def unit_test_case_module_dir_path(self) -> str:
        return str(
            Path('tests/unit') / Path(self.module_package.replace('.', '/')))
    
    @property
    def unit_test_case_module_path(self) -> str:
        return str(
            Path(self.unit_test_root_path) / Path(self.module_package.replace('.', '/') / Path(self.test_module_name)))

    def output(self):
        os.makedirs(os.path.dirname(self.unit_test_case_module_path), exist_ok=True)

        with open(self.unit_test_case_module_path, 'w') as fw:
            fw.write(self.test_case_content)

### <b><font color='darkgreen'>Prepare Testing Module</font></b>
Let's prepare the testing module for generation of unit test cases.

In [80]:
import my_math

help(my_math.add)

Help on function add in module my_math:

add(a: int, b: int) -> int
    Sums up the input `a` and `b`.

    Args:
      a: First value to add
      b: Second value to add

    Returns:
      Return value of `a + b`.



In [81]:
my_math.add(1, 2)

3

In [32]:
module_package = 'utils'
project_root_path = os.getcwd()
module_name = 'my_math'
my_math_module_content = open(
    Path(project_root_path) / module_package.replace('.', '/') / f'{module_name}.py', 'r').read()

test_module = TestModule(
    project_root_path=project_root_path,
    module_package = module_package,
    module_name=module_name,
    module_content=math_module_content,
)

In [208]:
test_module

TestModule(project_root_path='/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest', module_package='utils', module_name='my_math', module_content='\ndef add(a: int, b: int) -> int:\n    """Sums up the input `a` and `b`."""\n    return a + b\n')

### <b><font color='darkgreen'>Create Unit Test Cases</font></b>
Let's try to create unit test cases:

In [209]:
unit_test_case_result = lf.query(
    prompt='Please create unit test cases for {{module}}',
    schema=UnitTestCases,
    module=test_module,
    lm=model)

In [210]:
unit_test_case_result

UnitTestCases(project_root_path='/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest', module_package='utils', module_name='my_math', module_content='import unittest\nfrom utils.my_math import add\n\nclass TestAdd(unittest.TestCase):\n    def test_positive_numbers(self):\n        self.assertEqual(add(2, 3), 5)\n\n    def test_negative_numbers(self):\n        self.assertEqual(add(-2, -3), -5)\n\n    def test_zero(self):\n        self.assertEqual(add(0, 5), 5)\n        self.assertEqual(add(5, 0), 5)\n\n    def test_float(self):\n        self.assertEqual(add(2.5, 3.5), 6)\n')

In [84]:
unit_test_case_result.unit_test_case_module_path

'/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest/tests/unit/utils/test_my_math.py'

In [85]:
unit_test_case_result.unit_test_case_module_dir_path

'tests/unit/utils'

### <b><font color='darkgreen'>Execute Generated Unit Test Cases</font></b>
Below will demonstrate on how to execute the generated unit test cases:

#### <b>Output unit test case content into file</b>

In [63]:
# Output the unit test cases into file:
os.makedirs(os.path.dirname(unit_test_case_result.unit_test_case_module_path), exist_ok=True)

with open(unit_test_case_result.unit_test_case_module_path, 'w') as fw:
    fw.write(unit_test_case_result.module_content)

In [65]:
show_source_code(unit_test_case_result.unit_test_case_module_path)


```python
import unittest
from utils.my_math import add

class TestAdd(unittest.TestCase):
    def test_add_positive_numbers(self):
        self.assertEqual(add(1, 2), 3)

    def test_add_negative_numbers(self):
        self.assertEqual(add(-1, -2), -3)

    def test_add_zero(self):
        self.assertEqual(add(0, 0), 0)

    def test_add_positive_and_negative(self):
        self.assertEqual(add(1, -2), -1)
```

#### <b>Execute the generated unit test case file</b>

In [20]:
import unittest
import os
import sys

# Add the project root to the Python path to enable imports like 'utils.my_math'
# Assuming run_tests.py is at the root and tests/unit/utils/test_my_math.py is the test file
#project_root = unit_test_case_result.project_root_path
#sys.path.insert(0, project_root)

class MyCustomTestResult(unittest.TextTestResult):
    def __init__(self, stream, descriptions, verbosity):
        super().__init__(stream, descriptions, verbosity)
        # This is where 'all_test_results' is defined!
        self.all_test_results = []

    def addSuccess(self, test):
        super().addSuccess(test)
        self.all_test_results.append(f"{test} ... ok")

    def addFailure(self, test, err):
        super().addFailure(test, err)
        self.all_test_results.append(f"{test} ... FAILED")

    def addError(self, test, err):
        super().addError(test, err)
        self.all_test_results.append(f"{test} ... ERROR")

    def addSkip(self, test, reason):
        super().addSkip(test, reason)
        self.all_test_results.append(f"{test} ... SKIPPED")


class MyCustomTestRunner(unittest.TextTestRunner):
    def _makeResult(self):
        # This method tells the runner to use YOUR custom result class
        return MyCustomTestResult(self.stream, self.descriptions, self.verbosity)
        

class UnitTestRunner:
    def __init__(self, unit_test_case: UnitTestCases):
        self.unit_test_case = unit_test_case
        self._test_result = None

    def _generate_output(self, result) -> str:
        output_messages = []
        output_messages.append("--- Custom Test Output ---")
        for outcome_string in result.all_test_results:
            output_messages.append(outcome_string)

        output_messages.append('=' * 10)
        if result.wasSuccessful():
            output_messages.append("Overall Result: OK (All tests passed) ✅")
        else:
            output_messages.append("Overall Result: FAILED ❌")
        
        if result.failures:
            output_messages.append(f"Failures ({len(result.failures)}):")
            for test, traceback_str in result.failures:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")
        if result.errors:
            output_messages.append(f"Errors ({len(result.errors)}):")
            for test, traceback_str in result.errors:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")

        if result.skipped:
            output_messages.append(f"Skipped ({len(result.skipped)}):")
            for test, reason in result.skipped:
                output_messages.append(f"  Test: {test}, Reason: {reason}\n")
        if result.expectedFailures:
            output_messages.append(f"Expected Failures ({len(result.expectedFailures)}):")
            for test, traceback_str in result.expectedFailures:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")
        if result.unexpectedSuccesses:
            output_messages.append(f"Unexpected Successes ({len(result.unexpectedSuccesses)}):")
            for test in result.unexpectedSuccesses:
                output_messages.append(f"  Test: {test}\n")

        return '\n'.join(output_messages)
        
    def run(self) -> str:
        project_root = self.unit_test_case.project_root_path
        if project_root not in sys.path:
            sys.path.insert(0, project_root)
        
        # Output unit test cases into testing module.
        self.unit_test_case.output()
        
        # Discover tests from the 'tests' directory
        # start_dir: The directory to start searching for tests (e.g., 'tests')
        # pattern: Only look for files matching this pattern (e.g., 'test_*.py')
        test_loader = unittest.TestLoader()
        print(f'Start dir: {self.unit_test_case.unit_test_case_module_dir_path}')
        print(f'Test module name: {self.unit_test_case.test_module_name.split(".")[0]}')
        test_suite = test_loader.discover(
            start_dir=self.unit_test_case.unit_test_case_module_dir_path,
            pattern='test_*.py')
        #test_suite = test_loader.loadTestsFromName(
        #    self.unit_test_case.test_module_name.split('.')[0])

        # Run the tests
        runner = MyCustomTestRunner(verbosity=0) # verbosity=2 shows more details
        self._test_result = runner.run(test_suite)
        return self._generate_output(self._test_result)

In [19]:
sys.path

['/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest/tests/unit/utils/john',
 '/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest',
 '/usr/local/buildtools/current/sitecustomize',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.12/site-packages']

In [277]:
test_runner = UnitTestRunner(unit_test_case_result)

In [278]:
print(test_runner.run())

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK


Start dir: tests/unit/utils
Test module name: test_my_math
--- Custom Test Output ---
test_add_negative_numbers (test_my_math.TestAdd.test_add_negative_numbers) ... ok
test_add_positive_and_negative (test_my_math.TestAdd.test_add_positive_and_negative) ... ok
test_add_positive_numbers (test_my_math.TestAdd.test_add_positive_numbers) ... ok
test_add_zero (test_my_math.TestAdd.test_add_zero) ... ok
Overall Result: OK (All tests passed) ✅


### <b><font color='darkgreen'>More</font></b>

In [7]:
def create_test_module(
    module_package: str,
    module_name: str,
    project_root_path=None):
    project_root_path = project_root_path or os.getcwd()
    module_content = open(
        Path(project_root_path) / module_package.replace('.', '/') / f'{module_name}.py', 'r').read()

    return TestModule(
            project_root_path=project_root_path,
            module_package = module_package,
            module_name=module_name,
            module_content=module_content)

In [8]:
str_helper_module = create_test_module('utils.john', 'str_helper')

In [9]:
str_helper_module

TestModule(project_root_path='/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper', module_content='def to_upper(input_str: str) -> str:\n  """Turns input string into upper case."""\n  return input_str.upper()\n')

In [10]:
str_helper_unit_test_cases = lf.query(
    prompt='Please create unit test cases for {{module}}',
    schema=UnitTestCases,
    module=str_helper_module,
    lm=model)

In [11]:
str_helper_unit_test_cases

UnitTestCases(project_root_path='/usr/local/google/home/johnkclee/Github/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper', test_case_content='import unittest\nfrom utils.john.str_helper import to_upper\n\nclass TestToUpper(unittest.TestCase):\n    def test_empty_string(self):\n        self.assertEqual(to_upper(""), "")\n\n    def test_lower_case_string(self):\n        self.assertEqual(to_upper("hello"), "HELLO")\n\n    def test_mixed_case_string(self):\n        self.assertEqual(to_upper("hELLo"), "HELLO")\n\n    def test_already_upper_case_string(self):\n        self.assertEqual(to_upper("HELLO"), "HELLO")\n\n    def test_non_ascii_characters(self):\n      self.assertEqual(to_upper("你好"), "你好") # Non-ASCII should remain unchanged\n\n    def test_numeric_input(self):\n        with self.assertRaises(AttributeError): # Numbers do not have upper() methods\n            to_upper(123)\n\n    def test_none_input(self):\n        with self.assertRaises(AttributeError): # Check for None input\n            to_upper(None)\n\nif __name__ == "__main__":\n    unittest.main()')

In [17]:
runner = UnitTestRunner(str_helper_unit_test_cases)

In [275]:
#str_helper_unit_test_cases.output()

In [18]:
output = runner.run()
print(output)

----------------------------------------------------------------------
Ran 7 tests in 0.000s

OK


Start dir: tests/unit/utils/john
Test module name: test_str_helper
--- Custom Test Output ---
test_already_upper_case_string (test_str_helper.TestToUpper.test_already_upper_case_string) ... ok
test_empty_string (test_str_helper.TestToUpper.test_empty_string) ... ok
test_lower_case_string (test_str_helper.TestToUpper.test_lower_case_string) ... ok
test_mixed_case_string (test_str_helper.TestToUpper.test_mixed_case_string) ... ok
test_non_ascii_characters (test_str_helper.TestToUpper.test_non_ascii_characters) ... ok
test_none_input (test_str_helper.TestToUpper.test_none_input) ... ok
test_numeric_input (test_str_helper.TestToUpper.test_numeric_input) ... ok
Overall Result: OK (All tests passed) ✅


In [253]:
fix_str = lf.query(
    prompt='How to fix unit test cases  {{failed}} of {{module}} with outut as:' + output,
    #schema=UnitTestCases,
    module=str_helper_module,
    failed=str_helper_unit_test_cases,
    lm=model)

In [254]:
fix_str

'The issue lies within the escape sequences in your test string, specifically within the test case `test_string_with_special_characters`. You\'ve escaped the backslash (`\\\\`) correctly in most places, but missed it for the single quote (`\'`) and double quote (`"`) within the string.  Since these are inside a string delimited by double quotes, the single quote doesn\'t strictly *need* escaping, but the double quote does.  It\'s best practice to escape both for clarity and to avoid potential issues.\n\nHere\'s the corrected `TestToUpper` class within your `test_str_helper.py` file:\n\n```python\nimport unittest\nfrom utils.john.str_helper import to_upper\n\nclass TestToUpper(unittest.TestCase):\n\n    def test_empty_string(self):\n        self.assertEqual(to_upper(""), "")\n\n    def test_lowercase_string(self):\n        self.assertEqual(to_upper("hello"), "HELLO")\n\n    def test_mixed_case_string(self):\n        self.assertEqual(to_upper("hELLo"), "HELLO")\n\n    def test_already_up

In [255]:
str_helper_unit_test_cases_v2 = lf.query(
    prompt='Please follow {{fix}} to create unit test cases.',
    schema=UnitTestCases,
    fix=fix_str,
    lm=model)

In [257]:
str_helper_unit_test_cases_v2

UnitTestCases(project_root_path='', module_package='utils.john', module_name='str_helper', module_content='def to_upper(input_str):\n    if input_str is None:\n        raise AttributeError("Input cannot be None")\n    return input_str.upper()\n')

In [256]:
runner = UnitTestRunner(str_helper_unit_test_cases_v2)
output = runner.run()

----------------------------------------------------------------------
Ran 0 tests in 0.000s

NO TESTS RAN


Start dir: tests/unit/utils/john
Test module name: test_str_helper


In [301]:
#from utils.john import str_helper

test_loader = unittest.TestLoader()
#test_suite = test_loader.discover(
#    start_dir='tests/unit/utils/john')
    #pattern='test_*.py')
test_suite = test_loader.loadTestsFromName('test_str_helper')
runner = MyCustomTestRunner(verbosity=2) # verbosity=2 shows more details
test_result = runner.run(test_suite)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

NO TESTS RAN


## <b><font color='darkblue'>Supplement</font></b>
* [Colab - Langfun 101: Getting Started with Langfun](https://colab.research.google.com/github/google/langfun/blob/main/docs/notebooks/langfun101.ipynb)